# pre_Data-Format
### Step_1. FormatOfficial2Yolo

In [9]:
!python Code/pre_FormatOfficial2Yolo.py

1. Renaming Training Dataset_v5 --> Training_Dataset_v5
2. Backing-up Labels to Dataset/Training_Dataset_v5/labels_origin/...
100%|█████████████████████████████████████| 1000/1000 [00:00<00:00, 6317.84it/s]
3. Formatting Official Labels to yolo Type...
100%|███████████████████████████████████████| 1000/1000 [00:39<00:00, 25.58it/s]
4. Moving Labels to Dataset/Training_Dataset_v5/labels
100%|████████████████████████████████████| 1000/1000 [00:00<00:00, 15040.25it/s]
5. Renaming Dataset/Training_Dataset_v5/train --> Dataset/Training_Dataset_v5/images


# train_Model-Baseline
### Step_2. Baseline model

In [ ]:
!cd yolov5

